In [1]:
import networkx as nx         #pacchetto per reti
import numpy as np
import ipyparallel as ipp     #pacchetto per parallelizzazione
import time as clock
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize
import pickle
#import winsound
from Node import *            #pacchetto agente Node
from NetworkTool import *     #pacchetto rete
from GenerativeTool import *  #pacchetto generazione rete
from VoteTool import *        #pacchetto analisi voto
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def Core(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
     
    for i in range(Nodes):
        tfix=list(-2*np.random.random(FB)+1)
        tvar=list(-2*np.random.random(VB)+1)
        MG.add_node(Node(MG.number_of_nodes(),tfix,tvar, np.random.random(),TheTime)) #create nodes with FB fix belief and VB var beli
        
    TheTime=TheTime+1

    for i in range(MG.number_of_nodes()):
        tlay = np.random.randint(LayerC)
        MG.nodes()[i].aggF(TheTime)
        AttachFriendFirst(MG, MG.nodes()[i], np.random.randint(RndFr)+1, tlay, fbpos[tlay], fbdist[tlay], vbpos[tlay], vbdist[tlay],TheTime) #every node search for friends
    print(' Time ', TheTime)

In [3]:
def Grow(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
    for j in range(Iter): #creating and adding
        TheTime = TheTime+1
        LayerB = ((LayerA-LayerC)//(Iter-1))*j + LayerC
        NodesB = NodesA//Iter
        print('Iter ',j,' Layers ', LayerB, ' Time ', TheTime)
        for i in range(NodesB):
            tfix=list(-2*np.random.random(FB)+1)
            tvar=list(-2*np.random.random(VB)+1)
            tnew = Node(MG.number_of_nodes(), tfix, tvar, np.random.random(), TheTime) #create nodes with FB fix belief and VB var belief
            MG.add_node(tnew) 
            tnew.aggF(TheTime)
            tlay = np.random.randint(LayerB)
            AttachChoosenBelief(MG, tnew, np.random.randint(RndFr)+1, tlay, fbpos[tlay], fbdist[tlay], vbpos[tlay], vbdist[tlay], TheTime)

    print(MG.number_of_nodes())
    print(' Time ', TheTime)

In [ ]:
def Rand(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
    TheTime=TheTime+1 
    co=0
    while MG.number_of_nodes()<NodesA:
        tfix=list(-2*np.random.random(FB)+1)
        tvar=list(-2*np.random.random(VB)+1)
        tnew = Node(MG.number_of_nodes(), tfix, tvar, np.random.random(), TheTime) #create nodes with FB fix belief and VB var belief
        MG.add_node(tnew) 
        tnew.aggF(TheTime)
        
    for i in MG.nodes():
        if len(MG.neighbors(i))==0:
            co=co+1
            RandomLink(MG, i, LayerA, TheTime)
    print("added: ",co,'; percentage: ', (co/MG.number_of_nodes())*100,'% ')
    print(' Time ', TheTime)
    return(co/MG.number_of_nodes())

In [ ]:
AvgDist = {}
StdDist = {}

NodeList=[500,1000,1500,2000]
FBlist=[5]
VBlist=[10]
SDList=[0.5,1,2]
LayerList=[4,8,12]
InfluDist=[0.2]

AvgDist['mean'] = {k:[] for k in NodeList+SDList+FBlist+VBlist+LayerList+InfluDist}
StdDist['mean'] = {k:[] for k in NodeList+SDList+FBlist+VBlist+LayerList+InfluDist}
AvgDist['alay'] = {k:[] for k in NodeList+SDList+FBlist+VBlist+LayerList+InfluDist}
StdDist['alay'] = {k:[] for k in NodeList+SDList+FBlist+VBlist+LayerList+InfluDist}

print('\n', AvgDist,'\n',StdDist)
    
for rep in range(2):
    print('\n Repetition',rep)
    for NodesA in NodeList:    
        for SocDes in SDList:
            for FB in FBlist:
                for VB in VBlist:
                    for LayerA in LayerList:
                        for InDi in InfluDist:

                            #NodesA=random.randint(500,2500)
                            #yeah
                            print('\n Setup',NodesA,FB,VB,LayerA,InDi,SocDes)

                            MG = nx.MultiGraph()
                            TheTime = 0
                            RndFr=3
                            Nodes=50 #core set of 50 nodes
                            Iter=3
                            LayerC=2 #core layers
                            bd=0.05 #abstension strip
                            issue=np.random.random(FB+VB) #voting issue
                            Keys = [FB, VB, LayerA, InDi, SocDes, NodesA]
                            
                            print(' Time ', TheTime)


                            fbdist, fbpos, vbdist, vbpos, cbpos = [], [], [], [], []
                            for j in range(LayerA): #creates random vector in specified range
                                fbpos.append(list(np.random.randint(FB, size=np.random.randint(1,FB))))
                                fbdist.append(0.50*np.random.random()+0.2)
                            for j in range(LayerA): #creates random vector in specified range
                                vbpos.append(list(np.random.randint(VB, size=np.random.randint(1,VB))))
                                vbdist.append(0.50*np.random.random()+0.2)
                                cbpos.append(list(np.random.randint(VB, size=np.random.randint(1,VB))))
                            print('\n',fbdist,'\n',fbpos,'\n',vbdist,'\n',vbpos,'\n',cbpos)

                            Core(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime)
                            Grow(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime)
                            print(Rand(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime))

                            print(TheTime)

                            #AllVoteA(MG, issue)

                            BaseAnalisys(MG,LayerA)

                            EvolveInfluR(MG, 50000, cbpos, 0.75, InDi, TheTime, LayerA)
                            print(' Time ', TheTime)

                            AllVoteA(MG, issue)

                            voteMG=[]
                            for n in MG.nodes():
                                voteMG.append(n.getVote())

                            chLay=[]
                            i=0
                            while i < 3:  #choose tre layer
                                tr=random.randint(LayerA)
                                if tr not in chLay:
                                    chLay.append(tr)
                                    i=i+1
                                    
                            tmpAvg=[]
                            tmpStd=[]
                            for lay1 in chLay:

                                AllVoteA(MG, issue)

                                G1=ExtractLayer(MG,lay1)
                                Call1=100
                                Interviews1=[]
                                vote1=[]

                                for i in range(Call1):
                                    Interviews1.append(np.random.choice(G1.nodes()))

                                for n in Interviews1:
                                    SocialDesirability(n, G1, SocDes)
                                    vote1.append(n.getVote())

                                tmpAvg.append(np.mean(vote1))
                                tmpStd.append(np.std(vote1))

                            for key in Keys:
                                AvgDist['mean'][key].append(np.mean(tmpAvg)-np.mean(voteMG))
                                StdDist['mean'][key].append(np.mean(tmpStd)-np.std(voteMG))
                                for i in range(3):
                                    AvgDist['alay'][key].append(tmpAvg[i]-np.mean(voteMG))
                                    StdDist['alay'][key].append(tmpAvg[i]-np.std(voteMG))

                            MG.clear()
                            print('\n',AvgDist,'\n',StdDist,'\n------------------------------------------')
                        
                       
                        
    with open('SDPAD2.pickle', 'wb') as handle:
        pickle.dump(AvgDist, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('SDPSD2.pickle', 'wb') as handle:
        pickle.dump(StdDist, handle, protocol=pickle.HIGHEST_PROTOCOL)



 {'mean': {0.5: [], 1: [], 2: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.2: [], 1500: []}, 'alay': {0.5: [], 1: [], 2: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.2: [], 1500: []}} 
 {'mean': {0.5: [], 1: [], 2: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.2: [], 1500: []}, 'alay': {0.5: [], 1: [], 2: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.2: [], 1500: []}}

 Repetition 0

 Setup 500 5 10 4 0.2 0.5
 Time  0

 [0.36529378272265933, 0.36914921756785013, 0.5920700939188217, 0.3537012434397135] 
 [[0], [4, 3, 2], [2, 4, 2], [1, 3, 4, 0]] 
 [0.44100087221734313, 0.2150859703986126, 0.29317440736183725, 0.5003030180269525] 
 [[8, 2, 5, 5, 4], [3, 3, 3, 9, 0], [0, 5, 8, 9, 8, 2, 5], [3, 0, 5, 2, 2, 7, 6, 9, 2]] 
 [[6, 5, 2, 9, 7, 2], [1, 0, 8, 0, 4], [9, 3, 5], [2, 5, 5, 6, 1, 9]]
 Time  1
Iter  0  Layers  2  Time  1
Iter  1  Layers  3  Time  2
Iter  2  Layers  4  Time  3
548


C:\Users\Francesco\Anaconda4\lib\site-packages\networkx\algorithms\assortativity\correlation.py:285: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


 Time  0

 {'mean': {0.5: [-0.010025673980492287, -0.018060652398397072], 1: [], 2: [], 4: [-0.010025673980492287], 5: [-0.010025673980492287, -0.018060652398397072], 8: [-0.018060652398397072], 10: [-0.010025673980492287, -0.018060652398397072], 12: [], 2000: [], 1000: [], 500: [-0.010025673980492287, -0.018060652398397072], 0.2: [-0.010025673980492287, -0.018060652398397072], 1500: []}, 'alay': {0.5: [-0.040044209401513542, 0.010983810585737481, -0.0010166231257007947, -0.069289135279895553, 0.006265992197846286, 0.0088411858868580467], 1: [], 2: [], 4: [-0.040044209401513542, 0.010983810585737481, -0.0010166231257007947], 5: [-0.040044209401513542, 0.010983810585737481, -0.0010166231257007947, -0.069289135279895553, 0.006265992197846286, 0.0088411858868580467], 8: [-0.069289135279895553, 0.006265992197846286, 0.0088411858868580467], 10: [-0.040044209401513542, 0.010983810585737481, -0.0010166231257007947, -0.069289135279895553, 0.006265992197846286, 0.0088411858868580467], 12: [], 2

In [ ]:
print('\n',AvgDist)
print('\n', StdDist)

In [ ]:
#VoteAnalysis(MG, issue, bd,1)
#VoteALayerLight(MG, issue, bd, 0, LayerA)

In [ ]:
#PollLayers(MG, 100, 100, bd, issue, 1, 2, 1)

In [ ]:
#PlotLayer(MG,LayerA)

In [ ]:
funname = ['AvgDist','StdDist']
dictionary = [AvgDist,StdDist]
#n=0
setting = ['alay','mean']
#m=1
varname = ['SocialDesirability','Layers', 'Nodes']
varlist = [SDList, LayerList, NodeList]
#o=1

for n in range(len(dictionary)):
    for m in range(len(setting)):
        for o in range(len(varlist)):

            temp0 = []
            temp1 = []
            temp2 = []
            temp3 = []
            print('\n', funname[n], setting[m], varname[o],'\n')

            for j in varlist[o]:
                #print(j)
                temp0.append(dictionary[n][setting[m]][j])
                temp1.append(np.nanmean(dictionary[n][setting[m]][j]))
                temp2.append(np.nanmax(dictionary[n][setting[m]][j]))
                temp3.append(np.nanmin(dictionary[n][setting[m]][j]))

                print('value',j,', average',np.nanmean(dictionary[n][setting[m]][j]),', st dev',np.nanstd(dictionary[n][setting[m]][j]))

            plt.plot(varlist[o],temp0, 'kx')
            plt.plot(varlist[o],temp1, 'r-')
            plt.plot(varlist[o],temp2, 'g-')
            plt.plot(varlist[o],temp3, 'g-')
            plt.ylabel(funname[n])
            plt.xlabel(varname[o])
            plt.savefig('SDP2'+funname[n]+setting[m]+varname[o]+'.png',orientation='landscape',bbox_inches='tight',dpi='figure')
            plt.show()


In [ ]:
with open('SDPAD2.pickle', 'rb') as handle:
    AvgDist=pickle.load(handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('SDPSD2.pickle', 'rb') as handle:
    StdDist=pickle.load(handle, protocol=pickle.HIGHEST_PROTOCOL)